In [1]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import logging
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
!pip install scikit-learn
# !pip install shapely
# !pip install shap-hypetune
# !pip install xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix,precision_score
from sklearn.preprocessing import StandardScaler
# import shap
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
from sklearn.metrics import roc_auc_score
from scipy import stats
from xgboost import XGBClassifier
# from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
#Getting the context
contextId = 'Airline_performance'
context = Helpers.getOrCreateContext(contextId=contextId, localVars=locals())


inputDatasetParameter = Helpers.get_or_create_input_dataset(
    name="inputDataset",
    metadata=Metadata(input_name='Input Dataset', is_required=True, tooltip='Dataset to apply the transformation'),
    local_context=locals())
# inputDataset = Helpers.get_or_create_input_var(
#     name="inputDataset",
#     metadata=Metadata(input_name="Cleaned Dataset", is_required=True, default_value="incol", tooltip="Column to add"),
#     local_context=locals()
# )
Airline_Parameter = Helpers.get_or_create_input_var(name="Airline",
    metadata=Metadata(input_name="Airline", is_required=True, tooltip="All Airline names", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
    )

Departures_On_Time_Parameter = Helpers.get_or_create_input_var(
    name="Departures_On_Time",
    metadata=Metadata(input_name="Departures_On_Time", is_required=True, tooltip="All Departures_On_Time", multiple=False,datatypes=['STRING'] ,options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Departures_Delayed_Parameter = Helpers.get_or_create_input_var(
    name="Departures_Delayed",
    metadata=Metadata(input_name="Departures_Delayed", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

Sectors_Scheduled_Parameter = Helpers.get_or_create_input_var(
    name="Sectors_Scheduled",
    metadata=Metadata(input_name="Sectors_Scheduled", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Sectors_Flown_Parameter = Helpers.get_or_create_input_var(
    name="Sectors_Flown",
    metadata=Metadata(input_name="Sectors_Flown", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

# Airline_Parameter = Helpers.get_or_create_input_var(
#     name="Airline",
#     metadata=Metadata(input_name="Sectors_Flown", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
#     local_context=locals()
# )



Cancellations_Parameter = Helpers.get_or_create_input_var(
    name="Cancellations",
    metadata=Metadata(input_name="Cancellations", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Arrivals_Delayed_Parameter = Helpers.get_or_create_input_var(
    name="Arrivals_Delayed",
    metadata=Metadata(input_name="Arrivals_Delayed", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Departing_Port_Parameter = Helpers.get_or_create_input_var(
    name="Departing_Port",
    metadata=Metadata(input_name="Departing_Port", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

# useless_Parameter = Helpers.get_or_create_input_var(
#     name="useless",
#     metadata=Metadata(input_name="Departing_Port", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
#     local_context=locals()
# )



Arriving_Port_Parameter = Helpers.get_or_create_input_var(
    name="Arriving_Port",
    metadata=Metadata(input_name="Arriving_Port", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

Year_Parameter = Helpers.get_or_create_input_var(
    name="Year",
    metadata=Metadata(input_name="Year", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

Month_Num_Parameter = Helpers.get_or_create_input_var(
    name="Month_Num",
    metadata=Metadata(input_name="Month_Num", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)








outputDataset_Parameter =Helpers.get_or_create_output_dataset(
 name="outputDataset",
    metadata=Metadata(input_name='train', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)
    
    

outputDataset_Parameter2 =Helpers.get_or_create_output_dataset(
 name="outputDataset2",
    metadata=Metadata(input_name='test', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)
    


In [3]:
#Getting the parameters
airline=inputDatasetParameter.value

outputDataset=outputDataset_Parameter.value
outputDataset2=outputDataset_Parameter2.value

Airline = Airline_Parameter.value

Departures_On_Time =Departures_On_Time_Parameter.value
Departures_Delayed =    Departures_Delayed_Parameter.value
Sectors_Scheduled=    Sectors_Scheduled_Parameter.value
Sectors_Flown=    Sectors_Flown_Parameter.value
Cancellations=    Cancellations_Parameter.value
Month_Num = Month_Num_Parameter.value
Year = Year_Parameter.value
Arrivals_Delayed = Arrivals_Delayed_Parameter.value
Departing_Port=Departing_Port_Parameter.value
Arriving_Port=Arriving_Port_Parameter.value









# outputDataset2=outputDatasetParameter2.value

In [4]:
inputDatasetParameter.value


'airline'

In [5]:
airline =  Helpers.getEntityData(context, airline)

Exception: Entity with name airline doesn't exists, available entities are otp_time_series_web

In [9]:
airline.head(2)

,Route,Departing_Port,Arriving_Port,Airline,Month,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,Year,Month_Num
0,Adelaide-Brisbane,Adelaide,Brisbane,All Airlines,Jan-04,155.0,155,0.0,123.0,120.0,32.0,35.0,2004,1
1,Adelaide-Canberra,Adelaide,Canberra,All Airlines,Jan-04,75.0,75,0.0,72.0,72.0,3.0,3.0,2004,1


In [10]:
XTrain = airline.sample(frac=0.9,random_state=123)

In [12]:
XVal = airline.drop(XTrain.index)

In [14]:
len(XTrain),len(XVal)

(87574, 9730)

result= pd.DataFrame(model.predict(X_test_norm))

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset, dataFrame=XTrain)
out2 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset2, dataFrame=XVal)

# out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=Xtrain)
# out2 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset4, dataFrame=X_test_norm)
# out3 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset5, dataFrame=Y_train)
# out4 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset6, dataFrame=Y_test)
outputCollection.addTemplateOutput(out)
outputCollection.addTemplateOutput(out2)
# outputCollection.addTemplateOutput(out3)
# outputCollection.addTemplateOutput(out4)
Helpers.save(context)